In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
print(train.shape)
print(test.shape)


In [ ]:
x_train = train.iloc[:,1:].to_numpy().reshape(train.shape[0],28, 28)
y_train = train.label.to_numpy()
print(x_train.shape)
print(y_train.shape)

In [ ]:
#25 images
x_train, x_val, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=1234)
x_test = test.to_numpy().reshape(test.shape[0], 28,28)
print(x_test.shape)
print(x_val.shape)
print(x_train.shape)
print(y_train.shape)
print(y_train[0])
x_train, x_val, x_test = x_train/255, x_val/255, x_test/255
plt.figure(figsize=(10,5))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_train[i],cmap=plt.cm.binary)

In [ ]:
from tensorflow import keras
y_train_cat = keras.utils.to_categorical(y_train, 10)
y_test_cat = keras.utils.to_categorical(y_test, 10)


In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28,28,1)),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
])
print(model.summary())


In [ ]:
model.compile(optimizer='adam',  # optimizer
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
history=model.fit(x_train, y_train_cat, batch_size=32, epochs=50,validation_split=0.2,callbacks=[callback])
len(history.history['loss'])  #

In [ ]:
n=0
model.evaluate(x_val,y_test_cat)
x=np.expand_dims(x_test[n],axis=0)
res=model.predict(x)
print(res)
print(f"Recognized digit: {np.argmax(res)}")
plt.imshow(x_test[n],cmap=plt.cm.binary)


In [ ]:
pred=model.predict(x_test)
pred=np.argmax(pred,axis=1)


In [ ]:
from tensorflow.keras.utils import to_categorical
# reshape the dataset (Index,Height,Width,Channels)
X_train_cnn=train.iloc[:,1:].to_numpy().reshape(train.shape[0],28, 28,1)
X_test_cnn = test.to_numpy().reshape(test.shape[0], 28,28,1)


# Nomalize the dataset
X_train_cnn = X_train_cnn/255
X_test_cnn = X_test_cnn/255
# change the label to categorical
Y_train_cat_cnn = to_categorical(Y_train_cnn, num_classes=10)
# split the train data to train and validation data
x_train_cnn, x_val_cnn, y_train_cnn, y_val_cnn = train_test_split(X_train_cnn, 
                                                    Y_train_cnn, 
                                                    test_size=0.3, 
                                                    random_state=123)

print(x_train_cnn.shape)

from tensorflow.keras import  layers
cnn=tf.keras.Sequential([
        layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28,1)),
        layers.MaxPooling2D((3, 3)),
        
        layers.Conv2D(filters=10, kernel_size=(3, 3), activation='relu', input_shape=(28, 28,1)),
        layers.MaxPooling2D((2, 2)),
    
        tf.keras.layers.Flatten(),
        Dense(100, activation='relu'),
        Dense(10, activation='softmax')
])

cnn.compile(optimizer='adam',  # optimizer
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

print(x_train.shape)
history_cnn=cnn.fit(x_train_cnn, y_train_cnn, validation_data=(x_val_cnn,y_val_cnn),epochs=50,callbacks=[callback])
len(history_cnn.history['loss'])  #

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history_cnn for accuracy
plt.plot(history_cnn.history['accuracy'])
plt.plot(history_cnn.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history_cnn for loss
plt.plot(history_cnn.history['loss'])
plt.plot(history_cnn.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
"So as we can see that CNN performance is much better than ANN "

In [ ]:
print(X_test_cnn.shape)
print(x_val_cnn.shape)
pred_cnn=cnn.predict(X_test_cnn)
pred_cnn=np.argmax(pred,axis=0)


In [ ]:
submission=pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
print(submission.shape)
print(pred.shape)
submission['Label']=pred_cnn
submission.head()
submission.to_csv('submission.csv', index=False)